# Application: Interpolating Weak Reaction Rate Tables

The electron-capture reaction:

$${}^{56}\mathrm{Ni} + e^- \rightarrow {}^{56}\mathrm{Co} + \nu_e$$

is important in supernova.  A similiar transformation can happen via $\beta^+$ decay:

$${}^{56}\mathrm{Ni} \rightarrow {}^{56}\mathrm{Co} + e^+ + \nu_e$$

These rates are temperature and density dependent (in terms of electron density, $\rho Y_e$, where $Y_e$ is the electron fraction)
and evaluations of them are usually provided
as tables requiring reaction networks to interpolate.

## Accessing the data

Here's a table for the above rates: {download}`56ni-56co_electroncapture.dat` from [Langanke & Martínez-Pinedo 2001](https://www.sciencedirect.com/science/article/abs/pii/S0092640X01908654).  This has been cleaned up a little
from the tables provided there, combining both the e-capture and $\beta^+$-decay into a single rate.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [27]:
!head 56ni-56co_electroncapture.dat

#56ni -> 56co, e- capture
#Q=-1.624 MeV
#
#Log(rhoY)    Log(temp)     mu              dQ    Vs     Log(e-cap-rate)        Log(nu-energy-loss)  Log(gamma-energy)
#Log(g/cm^3)  Log(K)        erg             erg   erg    Log(1/s)               Log(erg/s)           Log(erg/s)
1.000000      7.000000      -4.806530e-09  0.00  0.00   -8.684000              -1.486129e+01         -100.00
1.000000      8.000000      -9.292624e-08  0.00  0.00   -9.164000              -1.533229e+01         -100.00
1.000000      8.301030      -2.146917e-07  0.00  0.00   -9.291000              -1.544729e+01         -100.00
1.000000      8.602060      -4.902661e-07  0.00  0.00   -9.387000              -1.551729e+01         -100.00
1.000000      8.845098      -8.058948e-07  0.00  0.00   -8.777000              -1.485829e+01         -100.00


We see that the data is in columns with most quantities stored as logs.  We're interest in the first column, $\log_{10}(\rho Y_e)$,
the second column, $\log_{10}(T)$, and the sixth column, $\log_{10}(\lambda)$ (the reaction rate).

```{tip}
We'll use `np.genfromtxt()` to read this, which can interpret a comment line as labels for the data, giving us an
easy way to index the columns.
```

In [28]:
data = np.genfromtxt("56ni-56co_electroncapture.dat", skip_header=3, names=True)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [29]:
ntemp = 13
nrho = 11

In [30]:
logrho = data["LogrhoY"][::ntemp]
logrho

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [31]:
logT = data["Logtemp"][0:ntemp]
logT

array([ 7.      ,  8.      ,  8.30103 ,  8.60206 ,  8.845098,  9.      ,
        9.176091,  9.30103 ,  9.477121,  9.69897 , 10.      , 10.477121,
       11.      ])

In [33]:
ec = data["Logecaprate"].reshape(nrho, ntemp)
ec[0:3, 0:3]

array([[-8.684, -9.164, -9.291],
       [-7.705, -8.165, -8.291],
       [-6.834, -7.171, -7.293]])

In [23]:
ec[0, :]

array([-8.684   , -9.164   , -9.291   , -9.387   , -8.777   , -7.439   ,
       -6.183988, -5.405627, -4.316341, -2.714602, -0.608944,  2.014581,
        4.728024])

## Bilinear interpolation

We'll do bilinear interpolation, using taking the 4 points in the table that surround the $(\rho Y_e, T)$ at which we wish to evaluate
the rate and fit it with a function:

$$\lambda(\rho Y_e, T) = a ((\rho Y_e) - (\rho Y_e)_i)(T - T_j) + b ((\rho Y_e) - (\rho Y_e)_i) + c (T - T_j) + d$$

We need to find the 4 unknowns, $\{a, b, c, d\}$, using the information from the table.

Visually, this appears as:

![bilinear interpolation](bilinear_ecapture.png)

where the $\times$ is where we want to know the rate.

## Using SciPy

In [34]:
from scipy import interpolate

In [36]:
interpolate.interpn((logrho, logT), ec, (1.5, 7.6), method="cubic")

array([-8.37241231])